In [ ]:
#H.6.1a
aux = 0
main = 1
n_bits = 2
dev = qml.device("default.qubit", wires=n_bits)

def add_two_unitaries(U, V):
    qml.Hadamard(wires=aux)
    qml.ControlledQubitUnitary(U,control_wires=[0], wires=1,control_values='0')
    qml.ControlledQubitUnitary(V,control_wires=[0], wires=1,control_values='1')
    qml.Hadamard(wires=aux)

In [ ]:
#H.6.1b
@qml.qnode(dev)
def X_plus_Z():
    U = qml.PauliX.compute_matrix()
    V = qml.PauliZ.compute_matrix()
    add_two_unitaries(U,V)
    return qml.state()

print("The amplitudes on the main register are proportional to", X_plus_Z()[:2], ".")

In [ ]:
#H.6.2
k_bits = 2
n_bits = 2
all_bits = k_bits + n_bits
aux = range(k_bits)
main = range(k_bits, all_bits)
dev = qml.device("default.qubit", wires=all_bits)

def SELECT_uniform(U_list):
    for index in range(2**k_bits):
        ctrl_str =  np.binary_repr(index, k_bits) # Create binary representation
        qml.ControlledQubitUnitary(U_list[index], control_wires =[i for i in range(k_bits)], wires = [i for i in range(n_bits, all_bits)], control_values=ctrl_str)

In [ ]:
#H.6.2b
@qml.qnode(dev)
def XH_plus_HZ():
    U_list = [np.kron(qml.PauliX.compute_matrix(), qml.PauliX.compute_matrix()),
              np.kron(qml.PauliZ.compute_matrix(), qml.PauliZ.compute_matrix()),
              np.kron(qml.PauliX.compute_matrix(), qml.PauliZ.compute_matrix()),
              np.kron(qml.PauliZ.compute_matrix(), qml.PauliX.compute_matrix())]
    qml.PauliX(wires=3)
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    SELECT_uniform(U_list)
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    return qml.state()

print("The amplitudes on the main register are proportional to", XH_plus_HZ()[:4], ".")

In [ ]:
#H.6.3
def V(t):
    return np.array(
        [
            [np.sqrt(t) / np.sqrt(t + 1), -1 / np.sqrt(t + 1)],
            [1 / np.sqrt(t + 1), np.sqrt(t) / np.sqrt(t + 1)],
        ]
    )

def exp_U_first(U, t):
    ml.QubitUnitary(V(t), wires=0)
    qml.ControlledQubitUnitary(U,control_wires=[0], wires=1, control_values='0')
    qml.ControlledQubitUnitary(np.identity(2),control_wires=[0], wires=1, control_values='0')
    qml.adjoint(qml.QubitUnitary)(V(t),wires=0)

In [ ]:
#H.6.4a
def prepare(coeffs):
    norm = np.linalg.norm(coeffs)
    normalized_coeffs = coeffs / norm
    qml.MottonenStatePreparation(normalized_coeffs, wires=aux)

In [ ]:
#H.6.4b
k_bits=2

def select(unitaries):
    for index in range(2**k_bits):
        ctrl_str = np.binary_repr(index, k_bits)[::-1]
        for i in range(len(ctrl_str)):
            if ctrl_str[i] == '0':
                qml.ControlledQubitUnitary(unitaries[i], control_wires=[j for j in aux], wires=[k for k in main], control_values='0')
            else:
                qml.ControlledQubitUnitary(unitaries[i], control_wires=[j for j in aux], wires=[k for k in main], control_values='1')

In [ ]:
#H.6.4c
def v0(t):
    v00 = 1
    v01 = np.sqrt(t)
    v02 = t/np.sqrt(2)
    v03 = 0
    return np.array([v00, v01, v02, v03])/np.sqrt(1+t+1/2*t**2)

def exp_U_second(unitaries, t):
    prepare(v0(t))
    select(unitaries)
    qml.adjoint(prepare(v0(t)))

In [ ]:
#H.6.5
aux = [0, 1]
main = [2]
all_bits = aux + main
U = qml.PauliX.compute_matrix()*(1.j)
unitaries = [np.eye(2), U, U @ U, np.eye(2)]
dev1 = qml.device("default.qubit", wires=[aux[0]] + main)
dev2 = qml.device("default.qubit", wires=all_bits)

# H.6.3
@qml.qnode(dev1)
def first_approx(t):
    exp_U_first(1j * qml.PauliX.compute_matrix(),t)
    return qml.state()

# H.6.4
@qml.qnode(dev2)
def second_approx(t):
    exp_U_second(unitaries, t)
    return qml.state()

@qml.qnode(dev2)
def exact(t):
    qml.RX(-2 * t,wires=main)
    return qml.state()